In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf().setAppName('Rollup_grouping').setMaster('yarn')
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print('app_id'.ljust(40), spark.sparkContext.applicationId)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/11 23:28:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/11 23:28:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/11 23:28:45 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
25/01/11 23:28:45 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


app_id                                   application_1727681258360_0102


In [3]:
from pyspark.sql.functions import grouping, sum, col, expr

df = spark.createDataFrame(
 [ ('Flight 1', 'Pax 1', 9), ('Flight 1', 'Pax 2', 16), 
       ('Flight 1', 'Pax 3', 20), ('Flight 1', 'Pax 4', 25),
   ('Flight 2', 'Pax 5', 6), ('Flight 2', 'Pax 6', 5), 
       ('Flight 2', 'Pax 7', 30), ('Flight 2', 'Pax 8', 5), 
   ('Flight 3', 'Pax 9', 6), ('Flight 3', 'Pax 10', 18), 
       ('Flight 3', 'Pax 11', 7), ('Flight 3', 'Pax 8', 4)], 
     ('flight', 'name', 'luggage_weight'))

df.rollup('flight', 'name').agg(
         sum('luggage_weight').alias('luggage_weight'),
         grouping('flight').alias('gr_flight_num'),
         grouping('name').alias('gr_name'),
         expr('case when sum(luggage_weight)>=50 \
               and gr_flight_num=0 and gr_name=1 then " Y" else "" end ').alias('luggage_overweight'))\
   .orderBy(col('flight').asc_nulls_last(), col('name').asc_nulls_last())\
   .na.fill('__TOTAL__')\
   .select('flight', 'name', 'luggage_weight', 'luggage_overweight').show(truncate=False)

+---------+---------+--------------+------------------+
|flight   |name     |luggage_weight|luggage_overweight|
+---------+---------+--------------+------------------+
|Flight 1 |Pax 1    |9             |                  |
|Flight 1 |Pax 2    |16            |                  |
|Flight 1 |Pax 3    |20            |                  |
|Flight 1 |Pax 4    |25            |                  |
|Flight 1 |__TOTAL__|70            | Y                |
|Flight 2 |Pax 5    |6             |                  |
|Flight 2 |Pax 6    |5             |                  |
|Flight 2 |Pax 7    |30            |                  |
|Flight 2 |Pax 8    |5             |                  |
|Flight 2 |__TOTAL__|46            |                  |
|Flight 3 |Pax 10   |18            |                  |
|Flight 3 |Pax 11   |7             |                  |
|Flight 3 |Pax 8    |4             |                  |
|Flight 3 |Pax 9    |6             |                  |
|Flight 3 |__TOTAL__|35            |            

In [4]:
# spark.stop()